# Now Featuring...

## Feature Detection with Open CV

From yesterday, we now know how image processing techniques can go transform an image from one thing into another. More importantly, we've seen how OpenCV itself handles this process. To get more into complex detection in OpenCV we will need a little more than changing pixels and converting images though. How do we go from just seeing edges to things like seeing corners or shapes? What about recognizing movement? For those, we need ways of determining how much of a change is made when we read an image, we need ways of modifying our processing behavior on a per pixel basis, we need so much more.

How much more complex can we go? Very. 

![https://docs.opencv.org/master/sift_keypoints.jpg](img/sift_keypoints.jpg)

Welcome to algorithmic world of feature detection.

Here we are go far beyond using one or two series of computations to transform our images. We have to start mixing methods, and using our transformations to determine information about the images themselves.This is described in series of alorithms, layered upon one another. A lot of this is the world of computer science. But we will take you through the basics so that you know the general concpets behind feature detection algorithms, how one of them might function, and an overview of several that are made available to you through OpenCV. 

In [ ]:
# Import the necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt  #image plotting
%matplotlib inline
import os #For interacting with Operating System - we use it for going through files
import glob #For file name pattern matching

## What on earth is a corner?

No really. What is a corner?

How do you imagine a corner vs an edge? What exactly lets us know that a corner is... what it is?

![https://docs.opencv.org/master/df/d54/tutorial_py_features_meaning.html](img/feature_building.jpg)

Looking at the above photo we can see the types of potential features that exist. But we need a way of describing them.

There is the A) the sky, B) wall texture, c) roof edge, D) another roof edge, and then E) and F) two corners. But how are these corners truly defined? Do they need to be this big? Can they be smaller or larger?  Which part of the image is the corner, or is it the whole thing?

Consider the following image

![person holding android phone in their hand](img/essential-phone-10.jpg)

The phone definitely has a _corner_, but maybe not as we would define it above. We have rounded edges and soft points. The natural world is full of things like this. Does the algorithm we devise have the ability to handle this?

This is an ongoing problem. We will see how scientists and algorithm designers have tackled it in the past.


## Playing the odds with histograms



It turns out a nice feature of images with pixel data is that they can be nicely binned into histograms. We can use those histograms, generated with kernels to determine, probabilistically what is inside of an image.

https://docs.opencv.org/master/d1/db7/tutorial_py_histogram_begins.html

These let us take all the pixels in a set and calculate which values of the pixels are the most common

In [ ]:
#taken roughly directly from the linked example
#this is demonstrated to show the basics of the process
img = cv.imread('img/not_a_pipe.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB);
imgplot = plt.imshow(img)
plt.show()
plt.hist(img.ravel(),256,[0,256]); plt.show()

## The Harris Corner Operation

Designed by Chris Harris and Mike Stephens
https://docs.opencv.org/master/dc/d0d/tutorial_py_features_harris.html

In very simple terms, this algorithm applies a sobel filter and then performs a histogram frequency counting operation. Depending on the values of that histogram, we determine if a pixel is past a threshold and worth keeping as a Corner feature.

In [ ]:
#this is the linked example

filename = 'img//not_a_pipe.jpg'
img = cv.imread(filename)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gray = np.float32(gray)
#harris takes multiple parameters, Image name, block size (neighborhood), ksize (for sobel), and then k, a constsance

dst = cv.cornerHarris(gray,2,3,0.04)
#result is dilated for marking the corners, not important
dst = cv.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]
cv.imshow('dst',img)


imgplot = plt.imshow(img)
plt.show()

print("note that it identifies mostly the lettering as corners and basically nothing on the pipe itself")

# Upgrades, people. Upgrades

We can do much, much more than the Harris operation.

The Scalar Invariant Feature Transform (SIFT) operation
https://docs.opencv.org/master/da/df5/tutorial_py_sift_intro.html

Harris did one through a pretty simple set of pathways. SIFT is here to do multiple. 

![SIFT explainer image](img/sift_explanation.jpg)

It is explained pretty well in the link above. But for brevity's sake, this algorithm does a similar operation to Harris, but Gaussian distributed instead of a simple neighborhood check, and also compares against several scales of the image to determine where a pixel might have a corner within it. If that doesn't make sense that's ok.

In [ ]:
#built in feature

img = cv.imread('img/not_a_pipe.jpg')
gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
sift = cv.SIFT_create()
kp = sift.detect(gray,None)
img=cv.drawKeypoints(gray,kp,img)
cv.imwrite('img/img_keypoints.jpg',img)

completedImage = cv.imread('img/img_keypoints.jpg', cv.COLOR_BGR2RGB);
imgplot = plt.imshow(completedImage)
plt.show()

print("Loook at how it captures far more detail about the potential corners in this image")

# "Open" Computer Vision with ORB

Algorithms and who they come from are vital to know. If this is your first encounter with sophisticated, layered processing techniques, you might be surprised to know that the SIFT algorithm is patented. You can't use it in your own software without paying for it.

The people behind the OpenCV library devised their own feature algorithm with ORB. This one is free, so you're good to go for your own projects using this algorithm!

https://docs.opencv.org/master/d1/d89/tutorial_py_orb.html

Designed by Ethan Rublee, Vincent Rabaud, Kurt Konolige and Gary R. Bradski. This is a prime example of the importance of knowing whose algorithms you are are using and whose perspectives you inject into your code. 

In [ ]:
#from the example

img = cv.imread('img/not_a_pipe.jpg',0)
# Initiate ORB detector
orb = cv.ORB_create()
# find the keypoints with ORB
kp = orb.detect(img,None)
# compute the descriptors with ORB
kp, des = orb.compute(img, kp)
# draw only keypoints location,not size and orientation
img2 = cv.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
plt.imshow(img2), plt.show()

## Summary

We have only scratched the surface of feature detection here. We could spend hours on implementation details, speculation about feature recognition, alternative image forms, and more, but there would be no way for us to fit that all into this workshop. 

Feature detection is an ongoing area of research, filled with opportunity for enterprising computational designers to make their mark. 

Remember that these algorithms are designed by people! They bear the marks of what a person thought about imagery. Those ideas might be durable and they might not be. Sometimes they get patented and stay that way for a long time. Sometimes a fast and free alternative is devised. Technologies change, Images change. Some algorithms perform better than others for certain purposes, some are completely beat by others. 

Algorithms are design, and now you know a little more about how to navigate them through your own work. 